In [7]:
import numpy as np
import math
from scipy.linalg import dft 

import numpy as np
import matplotlib.pyplot as plt
from os.path import dirname, abspath
import os
import matplotlib.pyplot as plt
from PIL import Image
import time

from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise

from scipy.linalg import circulant

In [2]:
def full_grad(z, meas, S):
    # Input: 
    #	- z, optimization iterate
    #   - meas, observed measurements
    # 	- M1, M2 precomputed matrices capturing subsampled blurred info
    # Output:
    # 	- Full gradient at z
    return S.T.dot(S.dot(z) - meas)

In [52]:
def stoch_grad(z, meas, S, mini_batch_size):
    # Input: 
    #	- z, optimization iterate
    #   - meas, observed measurements
    # 	- M1, M2 precomputed matrices capturing subsampled blurred info
    #   - ind, batch to update
    # Output:
    # 	- Full gradient at z

    # get parameters
    m = meas.size
    n = z.size
    idx = get_batch(mini_batch_size, m)
    print(idx.shape)
    res = S.dot(z) - meas
    print(res.shape)
    weights = np.multiply(idx,res)
    print(weights.shape)
    # output gradient as a matrix
    return S.T.dot(weights) 

In [47]:
def get_batch(B, N):
    # Input:
    # B, batch size
    # H, image height
    # W, image width
    # N, list of indices
    # Output:
    # batch indices

    #Generate random indices
    tmp = np.random.permutation(N)
    k = tmp[0:B]
    batch = np.zeros(N)
    batch[k] = 1

    # find nonzero batch indices
    return batch

In [34]:
np.random.seed(0)

## Set paths
data_path="../data/Set12/"
save_path="./figures/"
blur_path="./"
image_list = sorted(os.listdir(data_path))
blur_list = sorted(os.listdir(blur_path))

# set image and blurring kernel size
H = 64
W = 64

## Read test image
# normalize image pixels in range [0,1]
# orig = np.array(Image.open(data_path + image_list[8])) / 225.0
ORIG = Image.open(data_path + image_list[7]).resize((H,W))
ORIG = np.array(ORIG) / 255.0
mintmp = np.min(ORIG)
maxtmp = np.max(ORIG)
ORIG = (ORIG - mintmp) / (maxtmp - mintmp)

## Read blurring kernel
blur = Image.open(blur_path + blur_list[2]).resize((H,W))

## Set signal model parameters
H, W = ORIG.shape[:2] 				# height and width of orig image
N = H*W 							# image dimension
sigma = 100							# noise level of measurements

## Create noise
noises = np.random.normal(0, sigma, (N,))

## Create cirulant matrix
vb = np.matrix.flatten(np.asarray(blur)) # flatten blurring kernel into a vector
Cb = circulant(vb) # create circulant matrix of v_b
Cb_inv = np.linalg.inv(Cb)

## Vectorize orig image
vorig = np.matrix.flatten(np.asarray(ORIG))

s = 2 # subsampling rate
y0 = Cb.dot(vorig) + noises
y = y0[::s]
xinit = np.linalg.inv(Cb).dot(y0)

## Precompute essential matrices
idx = np.arange(0, H*W, s)
S_Cb = Cb[idx,:]
print(S_Cb.shape)
S_Cb_pinv = np.linalg.pinv(S_Cb)


(2048, 4096)


In [54]:
test_full_grad = full_grad(xinit, y, S_Cb)
test_stoch_grad = stoch_grad(xinit, y, S_Cb, 2048)
print(test_full_grad)
print(test_stoch_grad)

(2048,)
(2048,)
(2048,)
[-2.14277179e-06 -2.54106590e-06 -2.60645879e-06 ...  3.82724465e-07
 -4.27212399e-07 -1.44288242e-06]
[-2.14277179e-06 -2.54106590e-06 -2.60645879e-06 ...  3.82724465e-07
 -4.27212399e-07 -1.44288242e-06]


In [46]:
B = 10

#Generate random indices
tmp = np.random.permutation(N)
k = tmp[0:B]
batch = np.zeros(N)
batch[k] = 1

print(batch, batch[k])

[0. 0. 0. ... 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
